In [1]:

import networkx as nx
from networkx import ego_graph

import torch.optim as optim
import argparse
import numpy as np
import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv

from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

#from logger import Logger
from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader

In [2]:
dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
data = dataset[0]
#data.adj_t = data.adj_t.to_symmetric()
#data.adj_t = data.adj_t.to_symmetric()
print(data)
#split_idx = dataset.get_idx_split()
#train_idx = split_idx['train'].to(device)

Data(x=[2708, 1433], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], adj_t=[2708, 2708, nnz=10556])


In [3]:
train_index = np.where(data.train_mask)[0]
print(len(train_index))
valid_index = np.where(data.val_mask)[0]
print(len(valid_index))
test_index = np.where(data.test_mask)[0]
print(len(test_index))

140
500
1000


# GSAGE

In [3]:
import torch


class Logger(object):
    def __init__(self, runs, info=None):
        self.info = info
        self.results = [[] for _ in range(runs)]

    def add_result(self, run, result):
        assert len(result) == 3
        assert run >= 0 and run < len(self.results)
        self.results[run].append(result)

    def print_statistics(self, run=None):
        if run is not None:
            result = 100 * torch.tensor(self.results[run])
            argmax = result[:, 1].argmax().item()
            print(f'Run {run + 1:02d}:')
            print(f'Highest Train: {result[:, 0].max():.2f}')
            print(f'Highest Valid: {result[:, 1].max():.2f}')
            print(f'  Final Train: {result[argmax, 0]:.2f}')
            print(f'   Final Test: {result[argmax, 2]:.2f}')
        else:
            result = 100 * torch.tensor(self.results)

            best_results = []
            for r in result:
                train1 = r[:, 0].max().item()
                valid = r[:, 1].max().item()
                train2 = r[r[:, 1].argmax(), 0].item()
                test = r[r[:, 1].argmax(), 2].item()
                best_results.append((train1, valid, train2, test))

            best_result = torch.tensor(best_results)

            print(f'All runs:')
            r = best_result[:, 0]
            print(f'Highest Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 1]
            print(f'Highest Valid: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 2]
            print(f'  Final Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 3]
            print(f'   Final Test: {r.mean():.2f} ± {r.std():.2f}')

In [5]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x.log_softmax(dim=-1)


def train(model, data, train_idx, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)[train_idx]
    #print(len(out))
    #print(data.y.squeeze(1)[train_idx])
    loss = F.nll_loss(out, data.y.squeeze()[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()


def ACC(Prediction, Label):
    correct = Prediction.view(-1).eq(Label).sum().item()
    total=len(Label)
    return correct / total

@torch.no_grad()
def test(model, data, train_idx,valid_idx,test_idx):
    model.eval()

    out = model(data.x, data.adj_t)
    y_pred = out.argmax(dim=-1, keepdim=True)
    y_pred=y_pred.view(-1)
    train_acc=ACC(data.y[train_idx],y_pred[train_idx])
    valid_acc=ACC(data.y[valid_idx],y_pred[valid_idx])
    test_acc =ACC(data.y[test_idx],y_pred[test_idx])
    return train_acc, valid_acc, test_acc

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [7]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 50, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    
    train_idx = np.where(data.train_mask)[0]
    valid_idx = np.where(data.val_mask)[0]
    test_idx = np.where(data.test_mask)[0]
    
    model = SAGE(data.num_features, args.hidden_channels,
                    dataset.num_classes, args.num_layers,
                    args.dropout)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 2.1808, Train: 33.57%, Valid: 12.60% Test: 13.10%
Run: 01, Epoch: 02, Loss: 1.3327, Train: 57.14%, Valid: 22.00% Test: 21.10%
Run: 01, Epoch: 03, Loss: 1.0007, Train: 80.71%, Valid: 31.00% Test: 29.90%
Run: 01, Epoch: 04, Loss: 0.7840, Train: 95.00%, Valid: 40.40% Test: 40.30%
Run: 01, Epoch: 05, Loss: 0.6312, Train: 99.29%, Valid: 48.80% Test: 50.10%
Run: 01, Epoch: 06, Loss: 0.5327, Train: 100.00%, Valid: 55.00% Test: 55.30%
Run: 01, Epoch: 07, Loss: 0.4490, Train: 100.00%, Valid: 57.40% Test: 59.00%
Run: 01, Epoch: 08, Loss: 0.3303, Train: 100.00%, Valid: 57.60% Test: 61.00%
Run: 01, Epoch: 09, Loss: 0.3612, Train: 100.00%, Valid: 58.80% Test: 62.90%
Run: 01, Epoch: 10, Loss: 0.2454, Train: 100.00%, Valid: 60.20% Test: 63.80%
Run: 01, Epoch: 11, Loss: 0.2455, Train: 100.00%, Valid: 62.00% Test: 64.20%
Run: 01, Epoch: 12, Loss: 0.1936, Train: 100.00%, Valid: 62.40% Test: 64.70%
Run: 01, Epoch: 13, Loss: 0.1836, Train: 100.00%, Valid: 62.20% Test: 64.80%
Run:

Run: 03, Epoch: 07, Loss: 0.4381, Train: 99.29%, Valid: 63.60% Test: 68.80%
Run: 03, Epoch: 08, Loss: 0.2772, Train: 100.00%, Valid: 64.60% Test: 70.20%
Run: 03, Epoch: 09, Loss: 0.2962, Train: 100.00%, Valid: 65.80% Test: 70.80%
Run: 03, Epoch: 10, Loss: 0.2497, Train: 100.00%, Valid: 66.40% Test: 71.50%
Run: 03, Epoch: 11, Loss: 0.2367, Train: 100.00%, Valid: 66.80% Test: 71.70%
Run: 03, Epoch: 12, Loss: 0.1877, Train: 100.00%, Valid: 66.20% Test: 71.90%
Run: 03, Epoch: 13, Loss: 0.1220, Train: 100.00%, Valid: 66.60% Test: 72.00%
Run: 03, Epoch: 14, Loss: 0.1242, Train: 100.00%, Valid: 66.40% Test: 71.60%
Run: 03, Epoch: 15, Loss: 0.1030, Train: 100.00%, Valid: 67.20% Test: 71.30%
Run: 03, Epoch: 16, Loss: 0.1124, Train: 100.00%, Valid: 67.20% Test: 70.90%
Run: 03, Epoch: 17, Loss: 0.1056, Train: 100.00%, Valid: 66.80% Test: 70.70%
Run: 03, Epoch: 18, Loss: 0.0886, Train: 100.00%, Valid: 66.80% Test: 71.10%
Run: 03, Epoch: 19, Loss: 0.0709, Train: 100.00%, Valid: 67.00% Test: 71.10%


Run: 05, Epoch: 12, Loss: 0.2411, Train: 100.00%, Valid: 69.20% Test: 70.70%
Run: 05, Epoch: 13, Loss: 0.2432, Train: 100.00%, Valid: 69.00% Test: 71.50%
Run: 05, Epoch: 14, Loss: 0.2154, Train: 100.00%, Valid: 68.00% Test: 71.20%
Run: 05, Epoch: 15, Loss: 0.1812, Train: 100.00%, Valid: 67.60% Test: 71.20%
Run: 05, Epoch: 16, Loss: 0.1724, Train: 100.00%, Valid: 67.60% Test: 71.10%
Run: 05, Epoch: 17, Loss: 0.1588, Train: 100.00%, Valid: 67.80% Test: 71.00%
Run: 05, Epoch: 18, Loss: 0.1223, Train: 100.00%, Valid: 67.80% Test: 71.10%
Run: 05, Epoch: 19, Loss: 0.1184, Train: 100.00%, Valid: 67.80% Test: 70.80%
Run: 05, Epoch: 20, Loss: 0.1209, Train: 100.00%, Valid: 67.80% Test: 71.00%
Run: 05, Epoch: 21, Loss: 0.0950, Train: 100.00%, Valid: 68.00% Test: 71.10%
Run: 05, Epoch: 22, Loss: 0.0871, Train: 100.00%, Valid: 68.20% Test: 70.40%
Run: 05, Epoch: 23, Loss: 0.0980, Train: 100.00%, Valid: 68.00% Test: 70.20%
Run: 05, Epoch: 24, Loss: 0.0661, Train: 100.00%, Valid: 67.20% Test: 70.20%

Run: 07, Epoch: 17, Loss: 0.1122, Train: 100.00%, Valid: 65.80% Test: 69.50%
Run: 07, Epoch: 18, Loss: 0.1107, Train: 100.00%, Valid: 66.20% Test: 69.10%
Run: 07, Epoch: 19, Loss: 0.0790, Train: 100.00%, Valid: 65.40% Test: 69.30%
Run: 07, Epoch: 20, Loss: 0.0848, Train: 100.00%, Valid: 65.00% Test: 69.00%
Run: 07, Epoch: 21, Loss: 0.0865, Train: 100.00%, Valid: 65.00% Test: 68.80%
Run: 07, Epoch: 22, Loss: 0.0820, Train: 100.00%, Valid: 64.80% Test: 68.80%
Run: 07, Epoch: 23, Loss: 0.0373, Train: 100.00%, Valid: 64.40% Test: 68.60%
Run: 07, Epoch: 24, Loss: 0.0904, Train: 100.00%, Valid: 64.20% Test: 68.60%
Run: 07, Epoch: 25, Loss: 0.0712, Train: 100.00%, Valid: 64.00% Test: 68.70%
Run: 07, Epoch: 26, Loss: 0.0621, Train: 100.00%, Valid: 64.00% Test: 68.30%
Run: 07, Epoch: 27, Loss: 0.0675, Train: 100.00%, Valid: 64.00% Test: 68.00%
Run: 07, Epoch: 28, Loss: 0.0375, Train: 100.00%, Valid: 64.00% Test: 67.90%
Run: 07, Epoch: 29, Loss: 0.0465, Train: 100.00%, Valid: 64.20% Test: 68.00%

Run: 09, Epoch: 26, Loss: 0.0758, Train: 100.00%, Valid: 63.80% Test: 65.40%
Run: 09, Epoch: 27, Loss: 0.0528, Train: 100.00%, Valid: 63.80% Test: 65.50%
Run: 09, Epoch: 28, Loss: 0.0415, Train: 100.00%, Valid: 63.80% Test: 65.10%
Run: 09, Epoch: 29, Loss: 0.0644, Train: 100.00%, Valid: 63.60% Test: 65.20%
Run: 09, Epoch: 30, Loss: 0.0890, Train: 100.00%, Valid: 63.40% Test: 65.10%
Run: 09, Epoch: 31, Loss: 0.0650, Train: 100.00%, Valid: 63.60% Test: 65.40%
Run: 09, Epoch: 32, Loss: 0.0417, Train: 100.00%, Valid: 63.40% Test: 65.30%
Run: 09, Epoch: 33, Loss: 0.0501, Train: 100.00%, Valid: 63.40% Test: 65.40%
Run: 09, Epoch: 34, Loss: 0.0532, Train: 100.00%, Valid: 63.80% Test: 65.70%
Run: 09, Epoch: 35, Loss: 0.0495, Train: 100.00%, Valid: 64.00% Test: 66.10%
Run: 09, Epoch: 36, Loss: 0.0651, Train: 100.00%, Valid: 64.00% Test: 66.00%
Run: 09, Epoch: 37, Loss: 0.0525, Train: 100.00%, Valid: 64.40% Test: 66.20%
Run: 09, Epoch: 38, Loss: 0.0344, Train: 100.00%, Valid: 64.60% Test: 65.90%

# WISE EMbeddings

In [8]:
dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[2708, 1433], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], adj_t=[2708, 2708, nnz=10556])


In [8]:
import pandas as pd
Domain_Fec=pd.DataFrame(data.x.numpy())
label=pd.DataFrame(data.y.numpy(),columns =['class'])
Data=pd.concat([Domain_Fec,label], axis=1)
Data.head()

,0,1,2,3,4,5,6,7,8,9,...,1424,1425,1426,1427,1428,1429,1430,1431,1432,class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [9]:
Number_nodes=len(data.y)
fe_len=len(data.x[0])
catagories=Data['class'].to_numpy()
data_by_class = {cls: Data.loc[Data['class'] == cls].drop(['class'], axis=1) for cls in range(max(catagories) + 1)}
basis = [[max(df[i]) for i in range(len(df.columns))] for df in data_by_class.values()]
sel_basis = [[int(list(df[i].to_numpy()).count(1) >= int(len(df[i].index)*0.1)) 
              for i in range(len(df.columns))]
             for df in data_by_class.values()]
feature_names = [ii for ii in range(fe_len)]

In [ ]:
#It takes long time
Fec=[]
for i in range(23):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=np.zeros(7)
    for j in range(1433):
        for i in range(max(catagories)+1):
            if f[j]==1 and basis[i][j]==1:
                count[i]=count[i]+1;

    for i in range(max(catagories)+1):
        vec.append(count[i])
    f.clear()
    Fec.append(vec)
print(Fec)

In [11]:
Fec=[]
for i in range(Number_nodes):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    count5=0
    count6=0
    for j in range(fe_len):
        if f[j]==1 and basis[0][j]==1:
            count=count+1;
        if f[j]==1 and basis[1][j]==1:
            count1=count1+1;
        if f[j]==1 and basis[2][j]==1:
            count2=count2+1;
        if f[j]==1 and basis[3][j]==1:
            count3=count3+1;
        if f[j]==1 and basis[4][j]==1:
            count4=count4+1;
        if f[j]==1 and basis[5][j]==1:
            count5=count5+1;
        if f[j]==1 and basis[6][j]==1:
            count6=count6+1;
    vec.append(count)
    vec.append(count1)
    vec.append(count2)
    vec.append(count3)
    vec.append(count4)
    vec.append(count5)
    vec.append(count6)
    #print(f)
    f.clear()
    Fec.append(vec)

In [13]:
SFec=[]
for i in range(Number_nodes):
    Svec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    count5=0
    count6=0
    for j in range(1433):
        if f[j]==1 and sel_basis[0][j]==1:
            count=count+1;
        if f[j]==1 and sel_basis[1][j]==1:
            count1=count1+1;
        if f[j]==1 and sel_basis[2][j]==1:
            count2=count2+1;
        if f[j]==1 and sel_basis[3][j]==1:
            count3=count3+1;
        if f[j]==1 and sel_basis[4][j]==1:
            count4=count4+1;
        if f[j]==1 and sel_basis[5][j]==1:
            count5=count5+1;
        if f[j]==1 and sel_basis[6][j]==1:
            count6=count6+1;
    Svec.append(count)
    Svec.append(count1)
    Svec.append(count2)
    Svec.append(count3)
    Svec.append(count4)
    Svec.append(count5)
    Svec.append(count6)
    #print(f)
    f.clear()
    SFec.append(Svec)

In [15]:
Inc_fe=torch.tensor(Fec)
sel_fe=torch.tensor(SFec)
CC_domain=torch.cat((Inc_fe, sel_fe), 1).float()
print(CC_domain)
CC_domain.type()

tensor([[ 9.,  7.,  9.,  ...,  1.,  0.,  0.],
        [21., 20., 21.,  ...,  4.,  4.,  3.],
        [19., 17., 18.,  ...,  6.,  4.,  4.],
        ...,
        [16., 16., 15.,  ...,  3.,  5.,  3.],
        [12.,  9., 11.,  ...,  2.,  2.,  1.],
        [10., 12.,  8.,  ...,  2.,  1.,  1.]])


'torch.FloatTensor'

# W-GSAGE

In [16]:
data.x=CC_domain
print(data)

Data(x=[2708, 14], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], adj_t=[2708, 2708, nnz=10556])


In [17]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    
    train_idx = np.where(data.train_mask)[0]
    valid_idx = np.where(data.val_mask)[0]
    test_idx = np.where(data.test_mask)[0]
    
    model = SAGE(data.num_features, args.hidden_channels,
                    dataset.num_classes, args.num_layers,
                    args.dropout)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 2.0801, Train: 19.29%, Valid: 12.40% Test: 14.20%
Run: 01, Epoch: 02, Loss: 1.9698, Train: 17.86%, Valid: 13.20% Test: 14.30%
Run: 01, Epoch: 03, Loss: 1.8862, Train: 15.00%, Valid: 12.80% Test: 14.70%
Run: 01, Epoch: 04, Loss: 1.8016, Train: 15.71%, Valid: 14.40% Test: 16.00%
Run: 01, Epoch: 05, Loss: 1.7789, Train: 18.57%, Valid: 16.20% Test: 18.10%
Run: 01, Epoch: 06, Loss: 1.6890, Train: 24.29%, Valid: 17.20% Test: 19.90%
Run: 01, Epoch: 07, Loss: 1.6649, Train: 29.29%, Valid: 17.20% Test: 19.90%
Run: 01, Epoch: 08, Loss: 1.5283, Train: 31.43%, Valid: 16.00% Test: 19.40%
Run: 01, Epoch: 09, Loss: 1.5249, Train: 27.86%, Valid: 16.60% Test: 19.20%
Run: 01, Epoch: 10, Loss: 1.4811, Train: 31.43%, Valid: 17.60% Test: 19.90%
Run: 01, Epoch: 11, Loss: 1.4257, Train: 33.57%, Valid: 18.60% Test: 20.70%
Run: 01, Epoch: 12, Loss: 1.3160, Train: 36.43%, Valid: 21.80% Test: 23.50%
Run: 01, Epoch: 13, Loss: 1.2756, Train: 43.57%, Valid: 29.80% Test: 28.50%
Run: 01, Epo

Run: 01, Epoch: 127, Loss: 0.1916, Train: 79.29%, Valid: 63.00% Test: 64.80%
Run: 01, Epoch: 128, Loss: 0.2285, Train: 78.57%, Valid: 65.80% Test: 66.50%
Run: 01, Epoch: 129, Loss: 0.1663, Train: 81.43%, Valid: 69.80% Test: 70.30%
Run: 01, Epoch: 130, Loss: 0.2084, Train: 97.14%, Valid: 78.60% Test: 82.10%
Run: 01, Epoch: 131, Loss: 0.2061, Train: 96.43%, Valid: 78.60% Test: 82.80%
Run: 01, Epoch: 132, Loss: 0.1901, Train: 81.43%, Valid: 72.80% Test: 73.00%
Run: 01, Epoch: 133, Loss: 0.1802, Train: 70.71%, Valid: 63.40% Test: 65.10%
Run: 01, Epoch: 134, Loss: 0.2315, Train: 75.71%, Valid: 65.00% Test: 67.00%
Run: 01, Epoch: 135, Loss: 0.2113, Train: 84.29%, Valid: 73.40% Test: 73.80%
Run: 01, Epoch: 136, Loss: 0.1574, Train: 90.00%, Valid: 77.20% Test: 78.60%
Run: 01, Epoch: 137, Loss: 0.2054, Train: 94.29%, Valid: 76.80% Test: 80.00%
Run: 01, Epoch: 138, Loss: 0.1945, Train: 93.57%, Valid: 76.00% Test: 77.00%
Run: 01, Epoch: 139, Loss: 0.1870, Train: 95.71%, Valid: 76.80% Test: 78.70%

Run: 02, Epoch: 64, Loss: 0.3006, Train: 93.57%, Valid: 82.20% Test: 81.90%
Run: 02, Epoch: 65, Loss: 0.3513, Train: 92.86%, Valid: 80.80% Test: 81.40%
Run: 02, Epoch: 66, Loss: 0.3160, Train: 92.86%, Valid: 81.00% Test: 80.40%
Run: 02, Epoch: 67, Loss: 0.2759, Train: 95.71%, Valid: 80.00% Test: 81.20%
Run: 02, Epoch: 68, Loss: 0.3063, Train: 93.57%, Valid: 80.20% Test: 80.90%
Run: 02, Epoch: 69, Loss: 0.2656, Train: 92.14%, Valid: 79.20% Test: 82.30%
Run: 02, Epoch: 70, Loss: 0.3138, Train: 89.29%, Valid: 76.20% Test: 79.80%
Run: 02, Epoch: 71, Loss: 0.3003, Train: 87.86%, Valid: 76.60% Test: 78.90%
Run: 02, Epoch: 72, Loss: 0.3384, Train: 91.43%, Valid: 77.40% Test: 79.60%
Run: 02, Epoch: 73, Loss: 0.3264, Train: 94.29%, Valid: 81.60% Test: 83.30%
Run: 02, Epoch: 74, Loss: 0.2565, Train: 95.71%, Valid: 81.40% Test: 82.40%
Run: 02, Epoch: 75, Loss: 0.2588, Train: 96.43%, Valid: 82.00% Test: 82.60%
Run: 02, Epoch: 76, Loss: 0.2115, Train: 96.43%, Valid: 83.40% Test: 83.20%
Run: 02, Epo

Run: 03, Epoch: 02, Loss: 2.0619, Train: 27.86%, Valid: 15.00% Test: 15.50%
Run: 03, Epoch: 03, Loss: 1.9566, Train: 29.29%, Valid: 21.00% Test: 18.50%
Run: 03, Epoch: 04, Loss: 1.8374, Train: 33.57%, Valid: 30.00% Test: 24.40%
Run: 03, Epoch: 05, Loss: 1.7440, Train: 30.71%, Valid: 27.20% Test: 25.10%
Run: 03, Epoch: 06, Loss: 1.7196, Train: 21.43%, Valid: 20.80% Test: 19.00%
Run: 03, Epoch: 07, Loss: 1.6260, Train: 23.57%, Valid: 20.20% Test: 17.20%
Run: 03, Epoch: 08, Loss: 1.5577, Train: 30.00%, Valid: 22.00% Test: 19.40%
Run: 03, Epoch: 09, Loss: 1.5024, Train: 37.86%, Valid: 27.00% Test: 27.20%
Run: 03, Epoch: 10, Loss: 1.4862, Train: 47.86%, Valid: 31.60% Test: 33.00%
Run: 03, Epoch: 11, Loss: 1.4486, Train: 54.29%, Valid: 36.00% Test: 37.10%
Run: 03, Epoch: 12, Loss: 1.2734, Train: 55.00%, Valid: 36.40% Test: 37.60%
Run: 03, Epoch: 13, Loss: 1.3075, Train: 54.29%, Valid: 36.00% Test: 36.90%
Run: 03, Epoch: 14, Loss: 1.2251, Train: 55.00%, Valid: 36.40% Test: 37.30%
Run: 03, Epo

Run: 03, Epoch: 122, Loss: 0.2055, Train: 95.71%, Valid: 80.80% Test: 83.70%
Run: 03, Epoch: 123, Loss: 0.1432, Train: 95.71%, Valid: 82.80% Test: 84.40%
Run: 03, Epoch: 124, Loss: 0.1882, Train: 95.00%, Valid: 82.40% Test: 84.80%
Run: 03, Epoch: 125, Loss: 0.1726, Train: 91.43%, Valid: 82.60% Test: 82.50%
Run: 03, Epoch: 126, Loss: 0.1243, Train: 87.14%, Valid: 79.60% Test: 80.80%
Run: 03, Epoch: 127, Loss: 0.1640, Train: 91.43%, Valid: 81.00% Test: 82.20%
Run: 03, Epoch: 128, Loss: 0.1964, Train: 92.86%, Valid: 82.20% Test: 82.50%
Run: 03, Epoch: 129, Loss: 0.2121, Train: 89.29%, Valid: 76.60% Test: 78.10%
Run: 03, Epoch: 130, Loss: 0.1770, Train: 82.86%, Valid: 64.60% Test: 68.00%
Run: 03, Epoch: 131, Loss: 0.1421, Train: 75.71%, Valid: 59.00% Test: 59.90%
Run: 03, Epoch: 132, Loss: 0.2058, Train: 74.29%, Valid: 58.00% Test: 59.60%
Run: 03, Epoch: 133, Loss: 0.1462, Train: 78.57%, Valid: 61.20% Test: 62.80%
Run: 03, Epoch: 134, Loss: 0.1842, Train: 84.29%, Valid: 67.80% Test: 69.00%

...

Run: 08, Epoch: 48, Loss: 0.5017, Train: 78.57%, Valid: 67.60% Test: 68.10%
Run: 08, Epoch: 49, Loss: 0.4939, Train: 87.14%, Valid: 73.40% Test: 73.50%
Run: 08, Epoch: 50, Loss: 0.4679, Train: 91.43%, Valid: 78.20% Test: 79.70%
Run: 08, Epoch: 51, Loss: 0.5065, Train: 90.71%, Valid: 79.00% Test: 81.40%
Run: 08, Epoch: 52, Loss: 0.4417, Train: 86.43%, Valid: 78.40% Test: 77.40%
Run: 08, Epoch: 53, Loss: 0.4356, Train: 82.14%, Valid: 74.00% Test: 72.80%
Run: 08, Epoch: 54, Loss: 0.4001, Train: 80.00%, Valid: 71.80% Test: 74.30%
Run: 08, Epoch: 55, Loss: 0.4579, Train: 79.29%, Valid: 72.40% Test: 74.70%
Run: 08, Epoch: 56, Loss: 0.3403, Train: 82.14%, Valid: 69.80% Test: 71.60%
Run: 08, Epoch: 57, Loss: 0.3618, Train: 80.00%, Valid: 66.40% Test: 68.10%
Run: 08, Epoch: 58, Loss: 0.3861, Train: 78.57%, Valid: 61.00% Test: 62.40%
Run: 08, Epoch: 59, Loss: 0.3084, Train: 74.29%, Valid: 57.40% Test: 60.00%
Run: 08, Epoch: 60, Loss: 0.3983, Train: 76.43%, Valid: 59.20% Test: 61.80%
Run: 08, Epo

Run: 08, Epoch: 179, Loss: 0.1987, Train: 92.86%, Valid: 82.00% Test: 81.90%
Run: 08, Epoch: 180, Loss: 0.1382, Train: 97.86%, Valid: 84.80% Test: 84.60%
Run: 08, Epoch: 181, Loss: 0.1420, Train: 96.43%, Valid: 84.80% Test: 83.60%
Run: 08, Epoch: 182, Loss: 0.1295, Train: 92.86%, Valid: 82.40% Test: 82.70%
Run: 08, Epoch: 183, Loss: 0.1480, Train: 92.86%, Valid: 80.40% Test: 81.10%
Run: 08, Epoch: 184, Loss: 0.1583, Train: 92.14%, Valid: 80.40% Test: 80.00%
Run: 08, Epoch: 185, Loss: 0.1189, Train: 92.86%, Valid: 78.80% Test: 78.60%
Run: 08, Epoch: 186, Loss: 0.2192, Train: 89.29%, Valid: 74.20% Test: 76.00%
Run: 08, Epoch: 187, Loss: 0.1757, Train: 90.71%, Valid: 75.00% Test: 76.70%
Run: 08, Epoch: 188, Loss: 0.1422, Train: 92.14%, Valid: 75.80% Test: 76.70%
Run: 08, Epoch: 189, Loss: 0.1615, Train: 90.00%, Valid: 76.20% Test: 78.60%
Run: 08, Epoch: 190, Loss: 0.0932, Train: 90.00%, Valid: 78.20% Test: 78.70%
Run: 08, Epoch: 191, Loss: 0.1844, Train: 91.43%, Valid: 80.40% Test: 80.30%

Run: 09, Epoch: 110, Loss: 0.2547, Train: 78.57%, Valid: 72.20% Test: 74.10%
Run: 09, Epoch: 111, Loss: 0.2019, Train: 87.86%, Valid: 76.20% Test: 77.20%
Run: 09, Epoch: 112, Loss: 0.2605, Train: 85.71%, Valid: 67.40% Test: 69.30%
Run: 09, Epoch: 113, Loss: 0.2298, Train: 83.57%, Valid: 60.20% Test: 61.70%
Run: 09, Epoch: 114, Loss: 0.2153, Train: 84.29%, Valid: 60.40% Test: 61.90%
Run: 09, Epoch: 115, Loss: 0.2029, Train: 85.71%, Valid: 65.40% Test: 64.50%
Run: 09, Epoch: 116, Loss: 0.2802, Train: 84.29%, Valid: 68.60% Test: 67.10%
Run: 09, Epoch: 117, Loss: 0.2321, Train: 82.14%, Valid: 68.80% Test: 67.10%
Run: 09, Epoch: 118, Loss: 0.2614, Train: 77.14%, Valid: 68.60% Test: 66.90%
Run: 09, Epoch: 119, Loss: 0.2662, Train: 79.29%, Valid: 69.40% Test: 66.90%
Run: 09, Epoch: 120, Loss: 0.2399, Train: 78.57%, Valid: 73.40% Test: 72.00%
Run: 09, Epoch: 121, Loss: 0.2004, Train: 82.86%, Valid: 75.40% Test: 73.70%
Run: 09, Epoch: 122, Loss: 0.2209, Train: 81.43%, Valid: 76.20% Test: 77.10%

Run: 10, Epoch: 47, Loss: 0.4856, Train: 92.14%, Valid: 80.60% Test: 82.30%
Run: 10, Epoch: 48, Loss: 0.4664, Train: 91.43%, Valid: 78.60% Test: 82.10%
Run: 10, Epoch: 49, Loss: 0.3850, Train: 86.43%, Valid: 72.40% Test: 75.50%
Run: 10, Epoch: 50, Loss: 0.4064, Train: 83.57%, Valid: 69.20% Test: 70.30%
Run: 10, Epoch: 51, Loss: 0.4106, Train: 83.57%, Valid: 70.40% Test: 71.10%
Run: 10, Epoch: 52, Loss: 0.3352, Train: 87.14%, Valid: 72.80% Test: 75.50%
Run: 10, Epoch: 53, Loss: 0.4365, Train: 85.71%, Valid: 75.80% Test: 77.20%
Run: 10, Epoch: 54, Loss: 0.4138, Train: 86.43%, Valid: 75.60% Test: 77.50%
Run: 10, Epoch: 55, Loss: 0.3809, Train: 80.00%, Valid: 74.20% Test: 75.30%
Run: 10, Epoch: 56, Loss: 0.4103, Train: 79.29%, Valid: 72.80% Test: 73.40%
Run: 10, Epoch: 57, Loss: 0.3768, Train: 81.43%, Valid: 73.20% Test: 73.30%
Run: 10, Epoch: 58, Loss: 0.4189, Train: 85.71%, Valid: 76.00% Test: 75.50%
Run: 10, Epoch: 59, Loss: 0.4117, Train: 88.57%, Valid: 75.60% Test: 77.00%
Run: 10, Epo

Run: 10, Epoch: 185, Loss: 0.1542, Train: 87.86%, Valid: 75.60% Test: 75.60%
Run: 10, Epoch: 186, Loss: 0.1521, Train: 94.29%, Valid: 78.20% Test: 81.00%
Run: 10, Epoch: 187, Loss: 0.1487, Train: 97.14%, Valid: 82.20% Test: 84.80%
Run: 10, Epoch: 188, Loss: 0.1678, Train: 98.57%, Valid: 81.60% Test: 85.30%
Run: 10, Epoch: 189, Loss: 0.0968, Train: 97.14%, Valid: 81.40% Test: 83.10%
Run: 10, Epoch: 190, Loss: 0.1291, Train: 93.57%, Valid: 78.00% Test: 81.20%
Run: 10, Epoch: 191, Loss: 0.1551, Train: 94.29%, Valid: 77.40% Test: 80.80%
Run: 10, Epoch: 192, Loss: 0.1407, Train: 90.00%, Valid: 76.00% Test: 77.90%
Run: 10, Epoch: 193, Loss: 0.2330, Train: 91.43%, Valid: 77.00% Test: 78.40%
Run: 10, Epoch: 194, Loss: 0.0875, Train: 93.57%, Valid: 79.40% Test: 79.40%
Run: 10, Epoch: 195, Loss: 0.1241, Train: 95.00%, Valid: 80.00% Test: 79.00%
Run: 10, Epoch: 196, Loss: 0.1151, Train: 95.71%, Valid: 80.80% Test: 79.50%
Run: 10, Epoch: 197, Loss: 0.1677, Train: 96.43%, Valid: 81.00% Test: 80.80%

# Topological Encoding 

In [4]:
dataset = Planetoid(root='/tmp/cora', name='Cora')
data = dataset[0]
print(data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [5]:
print(data.edge_index.numpy())

[[ 633 1862 2582 ...  598 1473 2706]
 [   0    0    0 ... 2707 2707 2707]]


In [10]:
Edge_idx=data.edge_index.numpy()
Node=range(Number_nodes)
Edgelist=[]
for i in range(len(Edge_idx[1])):
    Edgelist.append((Edge_idx[0][i],Edge_idx[1][i]))
#print(Edgelist)

In [11]:
# a "plain" graph is undirected
G = nx.DiGraph()

# give each a node a 'name', which is a letter in this case.
#G.add_node('a')

# the add_nodes_from method allows adding nodes from a sequence, in this case a list
#nodes_to_add = ['b', 'c', 'd']
G.add_nodes_from(Node)

# add edge from 'a' to 'b'
# since this graph is undirected, the order doesn't matter here
#G.add_edge('a', 'b')

# just like add_nodes_from, we can add edges from a sequence
# edges should be specified as 2-tuples
#edges_to_add = [('a', 'c'), ('b', 'c'), ('c', 'd')]
G.add_edges_from(Edgelist)


In [12]:
print(G.number_of_edges())

10556


In [13]:
def Topological_Feature_subLevel(adj,filtration_fun, Filtration):
        betti_0=[]
        betti_1=[]
        for p in range(len(Filtration)):
            n_active = np.where(np.array(filtration_fun) <= Filtration[p])[0].tolist()
            Active_node=np.unique(n_active)
            if (len(Active_node)==0):
                betti_0.append(0)
                betti_1.append(0)
            else:
                b=adj[Active_node,:][:,Active_node]
                my_flag=pyflagser.flagser_unweighted(b, min_dimension=0, max_dimension=2, directed=False, coeff=2, approximation=None)
                x = my_flag["betti"]
                betti_0.append(x[0])
                betti_1.append(x[1])
            n_active.clear()
        return betti_0,betti_1

In [14]:
def Degree_list(Graph):
    degree_list = [Graph.degree(node) for node in Graph.nodes]
    return np.array(degree_list)

In [15]:
degree_list=Degree_list(G)
unique_list=np.unique(degree_list)
for d in unique_list:
    count=0
    for i in range(len(degree_list)):
        if degree_list[i]==d:
            count=count+1
    print(int(d)," | ",count,'\n')

2  |  485 

4  |  583 

6  |  553 

8  |  389 

10  |  281 

12  |  131 

14  |  82 

16  |  57 

18  |  25 

20  |  26 

22  |  14 

24  |  18 

26  |  5 

28  |  6 

30  |  6 

32  |  7 

34  |  8 

36  |  3 

38  |  5 

42  |  3 

44  |  1 

46  |  3 

52  |  1 

58  |  1 

60  |  2 

62  |  1 

64  |  2 

66  |  1 

68  |  1 

72  |  1 

80  |  1 

84  |  1 

88  |  1 

130  |  1 

148  |  1 

156  |  1 

336  |  1 



In [16]:
import pyflagser
Node_fil=[2,4,6,8,10,12,14,16,18,20,22,24,30,34]
topo_betti_0=[]
topo_betti_1=[]
Node_Edge=[]
for i in range(Number_nodes):
    print("\rProcessing file {} ({}%)".format(i, 100*i//(Number_nodes-1)), end='', flush=True)
    subgraph=ego_graph(G, i, radius=2, center=False, undirected=True, distance=None)
    filt=Degree_list(subgraph)
    A_sub = nx.to_numpy_array(subgraph)# adjacency matrix of subgraph
    fe=Topological_Feature_subLevel(A_sub,filt,Node_fil)
    topo_betti_0.append(fe[0])
    topo_betti_1.append(fe[1])
    Node_Edge.append([subgraph.number_of_nodes(),subgraph.number_of_edges()])
    #topo_with_NE.app

Processing file 2707 (100%)

In [28]:
dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[2708, 1433], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], adj_t=[2708, 2708, nnz=10556])


In [21]:
topo_betti0=torch.tensor(topo_betti_0).float()
topo_betti1=torch.tensor(topo_betti_1).float()
NodeEdge=torch.tensor(Node_Edge).float()

In [30]:
data.x=CC_domain
topo_fe=torch.cat((topo_betti0,topo_betti1),1)
data.topo=topo_fe
print(data)

Data(x=[2708, 14], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], adj_t=[2708, 2708, nnz=10556], topo=[2708, 28])


# TOPO-W-GSAGE

In [24]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        #return x.log_softmax(dim=-1)
        return x

class MLP(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters_mlp(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            #x = F.relu(x)
            x=F.sigmoid(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        #return torch.log_softmax(x, dim=-1)
        return x
    
class MLP2(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP2, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters_mlp2(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            #x = F.relu(x)
            x=F.sigmoid(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.log_softmax(x, dim=-1)
    

def train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2):
    model.train()
    mlp_model.train()
    mlp_2.train()
    optimizer.zero_grad()
    optimizer_mlp.zero_grad()
    optimizer_mlp2.zero_grad()
    gcn_embedding = model(data.x, data.adj_t)[train_idx]
    #print(gcn_embedding)
    mlp_embedding = mlp_model(data.topo[train_idx])
    #print(mlp_embedding)
    combined_embedding = torch.cat((gcn_embedding, mlp_embedding), dim=1)
    #print(combined_embedding)
    mlp_emb = mlp_2(combined_embedding)
    #print(mlp_emb)
    loss = F.nll_loss(mlp_emb, data.y.squeeze()[train_idx])
    #loss = F.nll_loss(combined_embedding, data.y.squeeze()[train_idx])
    loss.backward()
    optimizer_mlp2.step()
    optimizer.step()
    optimizer_mlp.step()
    

    return loss.item()


def ACC(Prediction, Label):
    correct = Prediction.view(-1).eq(Label).sum().item()
    total=len(Label)
    return correct / total



@torch.no_grad()
def test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx):
    model.eval()
    mlp_model.eval()
    mlp_2.eval()

    gcn_out = model(data.x, data.adj_t)
    #print(gcn_out[0])
    mlp_out=mlp_model(data.topo)
    #print(mlp_out)
    #out=torch.cat((gcn_out,mlp_out),dim=1)
    Com=torch.cat((gcn_out,mlp_out),dim=1)
    out=mlp_2(Com)
    y_pred = out.argmax(dim=-1, keepdim=True)
    #print(y_pred[0])
    y_pred=y_pred.view(-1)
    train_acc=ACC(data.y[train_idx],y_pred[train_idx])
    valid_acc=ACC(data.y[valid_idx],y_pred[valid_idx])
    test_acc =ACC(data.y[test_idx],y_pred[test_idx])
    return train_acc, valid_acc, test_acc

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [34]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01,'hidden_channels_mlp': 20,'dropout_mlp': 0.5,'num_layers_mlp': 3}

    args = objectview(args)
    #print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    X = data.x
    y_true = data.y
    data.adj_t = data.adj_t.to_symmetric()
    
    train_idx = np.where(data.train_mask)[0]
    valid_idx = np.where(data.val_mask)[0]
    test_idx = np.where(data.test_mask)[0]
    
    model = SAGE(data.num_features, args.hidden_channels,10, args.num_layers,args.dropout)
    mlp_model = MLP(X.size(-1), args.hidden_channels_mlp, 5,args.num_layers_mlp, args.dropout_mlp)
    #print(mlp_model.parameters())
    mlp_2 = MLP2(15, 100, dataset.num_classes,3, 0.0)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        model.reset_parameters()
        mlp_model.reset_parameters_mlp()
        mlp_2.reset_parameters_mlp2()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        optimizer_mlp=torch.optim.Adam(mlp_model.parameters(), lr=0.001)
        optimizer_mlp2=torch.optim.Adam(mlp_2.parameters(), lr=0.001)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2)
            result = test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

/Users/joshem/anaconda3/envs/tensorflow/lib/python3.10/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Run: 01, Epoch: 01, Loss: 1.9704, Train: 14.29%, Valid: 31.60% Test: 31.90%
Run: 01, Epoch: 02, Loss: 1.9240, Train: 15.71%, Valid: 31.80% Test: 31.90%
Run: 01, Epoch: 03, Loss: 1.9041, Train: 17.14%, Valid: 15.80% Test: 17.90%
Run: 01, Epoch: 04, Loss: 1.8681, Train: 17.14%, Valid: 7.20% Test: 7.80%
Run: 01, Epoch: 05, Loss: 1.8329, Train: 25.71%, Valid: 14.20% Test: 14.80%
Run: 01, Epoch: 06, Loss: 1.7852, Train: 20.71%, Valid: 12.00% Test: 12.30%
Run: 01, Epoch: 07, Loss: 1.7350, Train: 15.71%, Valid: 11.40% Test: 10.60%
Run: 01, Epoch: 08, Loss: 1.7458, Train: 21.43%, Valid: 11.60% Test: 11.60%
Run: 01, Epoch: 09, Loss: 1.7171, Train: 22.14%, Valid: 10.00% Test: 11.90%
Run: 01, Epoch: 10, Loss: 1.6710, Train: 26.43%, Valid: 9.60% Test: 12.70%
Run: 01, Epoch: 11, Loss: 1.6426, Train: 27.86%, Valid: 12.00% Test: 15.40%
Run: 01, Epoch: 12, Loss: 1.6208, Train: 32.14%, Valid: 12.60% Test: 17.30%
Run: 01, Epoch: 13, Loss: 1.6183, Train: 34.29%, Valid: 14.60% Test: 18.80%
Run: 01, Epoch:

Run: 01, Epoch: 121, Loss: 0.4513, Train: 92.14%, Valid: 79.00% Test: 80.00%
Run: 01, Epoch: 122, Loss: 0.4361, Train: 92.14%, Valid: 80.60% Test: 82.20%
Run: 01, Epoch: 123, Loss: 0.4255, Train: 93.57%, Valid: 80.40% Test: 81.80%
Run: 01, Epoch: 124, Loss: 0.4488, Train: 92.14%, Valid: 79.40% Test: 81.50%
Run: 01, Epoch: 125, Loss: 0.4406, Train: 92.86%, Valid: 79.00% Test: 82.30%
Run: 01, Epoch: 126, Loss: 0.4495, Train: 94.29%, Valid: 80.40% Test: 82.80%
Run: 01, Epoch: 127, Loss: 0.4468, Train: 93.57%, Valid: 80.40% Test: 82.40%
Run: 01, Epoch: 128, Loss: 0.4858, Train: 95.71%, Valid: 80.60% Test: 82.70%
Run: 01, Epoch: 129, Loss: 0.4432, Train: 94.29%, Valid: 80.20% Test: 82.20%
Run: 01, Epoch: 130, Loss: 0.4589, Train: 93.57%, Valid: 78.00% Test: 81.20%
Run: 01, Epoch: 131, Loss: 0.4311, Train: 93.57%, Valid: 78.60% Test: 80.10%
Run: 01, Epoch: 132, Loss: 0.4182, Train: 90.71%, Valid: 77.40% Test: 79.10%
Run: 01, Epoch: 133, Loss: 0.4663, Train: 92.86%, Valid: 81.00% Test: 81.70%

Run: 02, Epoch: 37, Loss: 1.0588, Train: 68.57%, Valid: 52.40% Test: 51.30%
Run: 02, Epoch: 38, Loss: 1.0139, Train: 71.43%, Valid: 54.40% Test: 52.90%
Run: 02, Epoch: 39, Loss: 1.0111, Train: 71.43%, Valid: 56.40% Test: 53.80%
Run: 02, Epoch: 40, Loss: 1.0322, Train: 72.86%, Valid: 58.80% Test: 56.00%
Run: 02, Epoch: 41, Loss: 1.0172, Train: 75.00%, Valid: 62.60% Test: 61.60%
Run: 02, Epoch: 42, Loss: 0.9813, Train: 81.43%, Valid: 68.00% Test: 70.60%
Run: 02, Epoch: 43, Loss: 0.9781, Train: 81.43%, Valid: 73.40% Test: 77.00%
Run: 02, Epoch: 44, Loss: 1.0076, Train: 80.00%, Valid: 75.00% Test: 75.10%
Run: 02, Epoch: 45, Loss: 0.9788, Train: 82.86%, Valid: 76.00% Test: 73.90%
Run: 02, Epoch: 46, Loss: 0.9285, Train: 87.14%, Valid: 77.20% Test: 75.40%
Run: 02, Epoch: 47, Loss: 0.9263, Train: 89.29%, Valid: 72.80% Test: 71.90%
Run: 02, Epoch: 48, Loss: 0.9474, Train: 78.57%, Valid: 63.40% Test: 62.00%
Run: 02, Epoch: 49, Loss: 0.8939, Train: 70.71%, Valid: 58.00% Test: 56.30%
Run: 02, Epo

Run: 02, Epoch: 159, Loss: 0.3306, Train: 84.29%, Valid: 68.00% Test: 69.40%
Run: 02, Epoch: 160, Loss: 0.3342, Train: 79.29%, Valid: 65.40% Test: 67.40%
Run: 02, Epoch: 161, Loss: 0.3183, Train: 83.57%, Valid: 66.80% Test: 69.70%
Run: 02, Epoch: 162, Loss: 0.3231, Train: 85.00%, Valid: 69.40% Test: 71.70%
Run: 02, Epoch: 163, Loss: 0.3293, Train: 91.43%, Valid: 75.00% Test: 76.00%
Run: 02, Epoch: 164, Loss: 0.3190, Train: 90.71%, Valid: 75.40% Test: 74.90%
Run: 02, Epoch: 165, Loss: 0.2808, Train: 82.86%, Valid: 71.20% Test: 70.80%
Run: 02, Epoch: 166, Loss: 0.3161, Train: 79.29%, Valid: 70.20% Test: 70.60%
Run: 02, Epoch: 167, Loss: 0.3071, Train: 80.00%, Valid: 73.00% Test: 69.00%
Run: 02, Epoch: 168, Loss: 0.3285, Train: 76.43%, Valid: 69.20% Test: 67.10%
Run: 02, Epoch: 169, Loss: 0.3235, Train: 81.43%, Valid: 69.80% Test: 68.50%
Run: 02, Epoch: 170, Loss: 0.2713, Train: 86.43%, Valid: 73.40% Test: 72.60%
Run: 02, Epoch: 171, Loss: 0.2945, Train: 89.29%, Valid: 78.00% Test: 78.10%

...

Run: 08, Epoch: 44, Loss: 0.9967, Train: 81.43%, Valid: 65.00% Test: 65.10%
Run: 08, Epoch: 45, Loss: 0.9699, Train: 78.57%, Valid: 60.60% Test: 62.10%
Run: 08, Epoch: 46, Loss: 0.9525, Train: 84.29%, Valid: 62.60% Test: 62.90%
Run: 08, Epoch: 47, Loss: 0.9489, Train: 84.29%, Valid: 64.20% Test: 64.30%
Run: 08, Epoch: 48, Loss: 0.9417, Train: 83.57%, Valid: 64.40% Test: 62.80%
Run: 08, Epoch: 49, Loss: 0.9126, Train: 78.57%, Valid: 57.20% Test: 59.40%
Run: 08, Epoch: 50, Loss: 0.9101, Train: 77.86%, Valid: 55.60% Test: 57.80%
Run: 08, Epoch: 51, Loss: 0.8744, Train: 80.00%, Valid: 54.40% Test: 57.20%
Run: 08, Epoch: 52, Loss: 0.9026, Train: 77.14%, Valid: 54.40% Test: 56.90%
Run: 08, Epoch: 53, Loss: 0.8656, Train: 75.00%, Valid: 57.60% Test: 57.70%
Run: 08, Epoch: 54, Loss: 0.8829, Train: 75.00%, Valid: 59.60% Test: 57.80%
Run: 08, Epoch: 55, Loss: 0.8504, Train: 75.00%, Valid: 60.80% Test: 57.50%
Run: 08, Epoch: 56, Loss: 0.8562, Train: 72.14%, Valid: 58.40% Test: 57.70%
Run: 08, Epo

Run: 08, Epoch: 160, Loss: 0.3133, Train: 82.14%, Valid: 74.60% Test: 72.70%
Run: 08, Epoch: 161, Loss: 0.3406, Train: 92.14%, Valid: 80.80% Test: 79.10%
Run: 08, Epoch: 162, Loss: 0.3081, Train: 97.14%, Valid: 83.60% Test: 81.80%
Run: 08, Epoch: 163, Loss: 0.3114, Train: 95.00%, Valid: 79.00% Test: 78.10%
Run: 08, Epoch: 164, Loss: 0.3521, Train: 86.43%, Valid: 71.40% Test: 69.00%
Run: 08, Epoch: 165, Loss: 0.3217, Train: 80.00%, Valid: 63.60% Test: 64.80%
Run: 08, Epoch: 166, Loss: 0.3430, Train: 80.00%, Valid: 64.40% Test: 64.70%
Run: 08, Epoch: 167, Loss: 0.3176, Train: 83.57%, Valid: 67.20% Test: 67.10%
Run: 08, Epoch: 168, Loss: 0.3028, Train: 86.43%, Valid: 67.20% Test: 67.40%
Run: 08, Epoch: 169, Loss: 0.3274, Train: 90.71%, Valid: 73.60% Test: 72.80%
Run: 08, Epoch: 170, Loss: 0.2999, Train: 93.57%, Valid: 77.60% Test: 78.40%
Run: 08, Epoch: 171, Loss: 0.2639, Train: 94.29%, Valid: 78.60% Test: 80.30%
Run: 08, Epoch: 172, Loss: 0.3384, Train: 90.71%, Valid: 80.00% Test: 80.60%

Run: 09, Epoch: 76, Loss: 0.6941, Train: 92.86%, Valid: 86.00% Test: 85.80%
Run: 09, Epoch: 77, Loss: 0.7059, Train: 92.14%, Valid: 83.60% Test: 84.80%
Run: 09, Epoch: 78, Loss: 0.7166, Train: 87.86%, Valid: 82.40% Test: 81.00%
Run: 09, Epoch: 79, Loss: 0.7031, Train: 90.00%, Valid: 82.20% Test: 81.30%
Run: 09, Epoch: 80, Loss: 0.6872, Train: 92.14%, Valid: 82.80% Test: 81.70%
Run: 09, Epoch: 81, Loss: 0.6956, Train: 87.14%, Valid: 75.40% Test: 75.50%
Run: 09, Epoch: 82, Loss: 0.7199, Train: 77.86%, Valid: 65.40% Test: 64.40%
Run: 09, Epoch: 83, Loss: 0.6627, Train: 85.71%, Valid: 71.80% Test: 69.40%
Run: 09, Epoch: 84, Loss: 0.6637, Train: 92.86%, Valid: 83.40% Test: 81.70%
Run: 09, Epoch: 85, Loss: 0.6578, Train: 92.86%, Valid: 85.20% Test: 82.40%
Run: 09, Epoch: 86, Loss: 0.6610, Train: 86.43%, Valid: 78.20% Test: 77.20%
Run: 09, Epoch: 87, Loss: 0.5967, Train: 78.57%, Valid: 69.80% Test: 69.30%
Run: 09, Epoch: 88, Loss: 0.6239, Train: 76.43%, Valid: 67.40% Test: 66.70%
Run: 09, Epo

Run: 09, Epoch: 192, Loss: 0.2506, Train: 87.14%, Valid: 70.80% Test: 68.80%
Run: 09, Epoch: 193, Loss: 0.2639, Train: 85.00%, Valid: 72.00% Test: 68.80%
Run: 09, Epoch: 194, Loss: 0.2537, Train: 88.57%, Valid: 74.60% Test: 73.60%
Run: 09, Epoch: 195, Loss: 0.2984, Train: 95.71%, Valid: 78.20% Test: 77.20%
Run: 09, Epoch: 196, Loss: 0.2834, Train: 92.86%, Valid: 75.20% Test: 72.90%
Run: 09, Epoch: 197, Loss: 0.2214, Train: 87.14%, Valid: 68.20% Test: 67.90%
Run: 09, Epoch: 198, Loss: 0.3181, Train: 84.29%, Valid: 62.60% Test: 62.20%
Run: 09, Epoch: 199, Loss: 0.2784, Train: 82.14%, Valid: 59.60% Test: 61.90%
Run: 09, Epoch: 200, Loss: 0.3052, Train: 85.00%, Valid: 69.00% Test: 68.40%
Run 09:
Highest Train: 97.86
Highest Valid: 86.00
  Final Train: 92.86
   Final Test: 85.80
Run: 10, Epoch: 01, Loss: 1.9953, Train: 11.43%, Valid: 7.20% Test: 7.30%
Run: 10, Epoch: 02, Loss: 1.9454, Train: 14.29%, Valid: 7.00% Test: 7.90%
Run: 10, Epoch: 03, Loss: 1.9088, Train: 16.43%, Valid: 9.20% Test:

Run: 10, Epoch: 108, Loss: 0.5195, Train: 94.29%, Valid: 76.00% Test: 77.20%
Run: 10, Epoch: 109, Loss: 0.5164, Train: 91.43%, Valid: 77.40% Test: 78.50%
Run: 10, Epoch: 110, Loss: 0.4966, Train: 91.43%, Valid: 80.00% Test: 80.70%
Run: 10, Epoch: 111, Loss: 0.4825, Train: 95.00%, Valid: 81.80% Test: 83.80%
Run: 10, Epoch: 112, Loss: 0.5071, Train: 93.57%, Valid: 80.00% Test: 84.10%
Run: 10, Epoch: 113, Loss: 0.4597, Train: 92.86%, Valid: 78.20% Test: 82.20%
Run: 10, Epoch: 114, Loss: 0.4525, Train: 87.86%, Valid: 75.80% Test: 75.10%
Run: 10, Epoch: 115, Loss: 0.4843, Train: 85.71%, Valid: 74.00% Test: 72.60%
Run: 10, Epoch: 116, Loss: 0.4685, Train: 85.00%, Valid: 73.60% Test: 73.30%
Run: 10, Epoch: 117, Loss: 0.4824, Train: 81.43%, Valid: 74.80% Test: 73.50%
Run: 10, Epoch: 118, Loss: 0.4483, Train: 80.00%, Valid: 74.40% Test: 72.90%
Run: 10, Epoch: 119, Loss: 0.5054, Train: 80.71%, Valid: 73.60% Test: 70.80%
Run: 10, Epoch: 120, Loss: 0.4739, Train: 86.43%, Valid: 79.20% Test: 76.10%

# TOPO-GSAGE

In [25]:
dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
data = dataset[0]
topo_fe=torch.cat((topo_betti0,topo_betti1),1)
data.topo=topo_fe
print(data)


Data(x=[2708, 1433], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], adj_t=[2708, 2708, nnz=10556], topo=[2708, 28])


In [26]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01,'hidden_channels_mlp': 20,'dropout_mlp': 0.5,'num_layers_mlp': 3}

    args = objectview(args)
    #print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    X = data.topo
    y_true = data.y
    data.adj_t = data.adj_t.to_symmetric()
    
    train_idx = np.where(data.train_mask)[0]
    valid_idx = np.where(data.val_mask)[0]
    test_idx = np.where(data.test_mask)[0]
    
    model = SAGE(data.num_features, args.hidden_channels,10, args.num_layers,args.dropout)
    mlp_model = MLP(X.size(-1), args.hidden_channels_mlp, 5,args.num_layers_mlp, args.dropout_mlp)
    #print(mlp_model.parameters())
    mlp_2 = MLP2(15, 100, dataset.num_classes,3, 0.0)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        model.reset_parameters()
        mlp_model.reset_parameters_mlp()
        mlp_2.reset_parameters_mlp2()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        optimizer_mlp=torch.optim.Adam(mlp_model.parameters(), lr=0.001)
        optimizer_mlp2=torch.optim.Adam(mlp_2.parameters(), lr=0.001)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2)
            result = test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 1.9624, Train: 14.29%, Valid: 16.20% Test: 14.90%
Run: 01, Epoch: 02, Loss: 1.8239, Train: 14.29%, Valid: 16.20% Test: 14.90%
Run: 01, Epoch: 03, Loss: 1.7632, Train: 15.00%, Valid: 16.20% Test: 14.90%
Run: 01, Epoch: 04, Loss: 1.7114, Train: 25.71%, Valid: 17.40% Test: 16.80%
Run: 01, Epoch: 05, Loss: 1.6508, Train: 27.14%, Valid: 19.80% Test: 18.70%
Run: 01, Epoch: 06, Loss: 1.5860, Train: 49.29%, Valid: 21.60% Test: 21.10%
Run: 01, Epoch: 07, Loss: 1.5398, Train: 42.14%, Valid: 13.20% Test: 15.30%
Run: 01, Epoch: 08, Loss: 1.5019, Train: 44.29%, Valid: 14.40% Test: 15.40%
Run: 01, Epoch: 09, Loss: 1.4348, Train: 50.71%, Valid: 15.20% Test: 17.10%
Run: 01, Epoch: 10, Loss: 1.3976, Train: 55.71%, Valid: 18.80% Test: 19.10%
Run: 01, Epoch: 11, Loss: 1.3674, Train: 62.86%, Valid: 22.00% Test: 21.70%
Run: 01, Epoch: 12, Loss: 1.3163, Train: 67.86%, Valid: 26.80% Test: 26.90%
Run: 01, Epoch: 13, Loss: 1.2866, Train: 70.71%, Valid: 31.40% Test: 31.60%
Run: 01, Epo

Run: 01, Epoch: 111, Loss: 0.2359, Train: 100.00%, Valid: 74.40% Test: 76.00%
Run: 01, Epoch: 112, Loss: 0.2440, Train: 100.00%, Valid: 74.20% Test: 76.40%
Run: 01, Epoch: 113, Loss: 0.2694, Train: 100.00%, Valid: 74.40% Test: 76.30%
Run: 01, Epoch: 114, Loss: 0.2360, Train: 100.00%, Valid: 74.60% Test: 76.10%
Run: 01, Epoch: 115, Loss: 0.2398, Train: 100.00%, Valid: 74.40% Test: 76.00%
Run: 01, Epoch: 116, Loss: 0.2535, Train: 100.00%, Valid: 74.00% Test: 76.00%
Run: 01, Epoch: 117, Loss: 0.2447, Train: 100.00%, Valid: 74.20% Test: 75.80%
Run: 01, Epoch: 118, Loss: 0.2343, Train: 100.00%, Valid: 74.60% Test: 75.70%
Run: 01, Epoch: 119, Loss: 0.2542, Train: 100.00%, Valid: 74.00% Test: 75.90%
Run: 01, Epoch: 120, Loss: 0.2235, Train: 100.00%, Valid: 73.40% Test: 75.80%
Run: 01, Epoch: 121, Loss: 0.2168, Train: 100.00%, Valid: 73.40% Test: 75.60%
Run: 01, Epoch: 122, Loss: 0.2033, Train: 100.00%, Valid: 73.00% Test: 75.50%
Run: 01, Epoch: 123, Loss: 0.2232, Train: 100.00%, Valid: 73.00%

Run: 02, Epoch: 16, Loss: 1.1575, Train: 100.00%, Valid: 72.80% Test: 76.30%
Run: 02, Epoch: 17, Loss: 1.1379, Train: 100.00%, Valid: 74.80% Test: 77.30%
Run: 02, Epoch: 18, Loss: 1.1111, Train: 100.00%, Valid: 74.80% Test: 78.10%
Run: 02, Epoch: 19, Loss: 1.0673, Train: 100.00%, Valid: 75.00% Test: 78.40%
Run: 02, Epoch: 20, Loss: 1.0621, Train: 100.00%, Valid: 75.60% Test: 78.40%
Run: 02, Epoch: 21, Loss: 1.0204, Train: 100.00%, Valid: 76.60% Test: 79.10%
Run: 02, Epoch: 22, Loss: 1.0141, Train: 100.00%, Valid: 77.40% Test: 79.50%
Run: 02, Epoch: 23, Loss: 0.9930, Train: 100.00%, Valid: 77.80% Test: 79.20%
Run: 02, Epoch: 24, Loss: 0.9770, Train: 100.00%, Valid: 77.00% Test: 79.40%
Run: 02, Epoch: 25, Loss: 0.9741, Train: 100.00%, Valid: 77.00% Test: 79.50%
Run: 02, Epoch: 26, Loss: 0.9486, Train: 100.00%, Valid: 76.80% Test: 79.20%
Run: 02, Epoch: 27, Loss: 0.9374, Train: 100.00%, Valid: 76.80% Test: 79.30%
Run: 02, Epoch: 28, Loss: 0.9212, Train: 100.00%, Valid: 77.00% Test: 78.90%

Run: 02, Epoch: 125, Loss: 0.2209, Train: 100.00%, Valid: 73.80% Test: 76.30%
Run: 02, Epoch: 126, Loss: 0.2061, Train: 100.00%, Valid: 74.00% Test: 76.20%
Run: 02, Epoch: 127, Loss: 0.2188, Train: 100.00%, Valid: 74.20% Test: 76.30%
Run: 02, Epoch: 128, Loss: 0.1978, Train: 100.00%, Valid: 74.40% Test: 75.70%
Run: 02, Epoch: 129, Loss: 0.2033, Train: 100.00%, Valid: 74.00% Test: 75.50%
Run: 02, Epoch: 130, Loss: 0.2095, Train: 100.00%, Valid: 73.80% Test: 75.50%
Run: 02, Epoch: 131, Loss: 0.2147, Train: 100.00%, Valid: 73.60% Test: 75.70%
Run: 02, Epoch: 132, Loss: 0.2196, Train: 100.00%, Valid: 73.60% Test: 75.90%
Run: 02, Epoch: 133, Loss: 0.1988, Train: 100.00%, Valid: 73.40% Test: 75.90%
Run: 02, Epoch: 134, Loss: 0.2092, Train: 100.00%, Valid: 73.40% Test: 75.90%
Run: 02, Epoch: 135, Loss: 0.2093, Train: 100.00%, Valid: 73.60% Test: 75.90%
Run: 02, Epoch: 136, Loss: 0.1998, Train: 100.00%, Valid: 73.60% Test: 75.80%
Run: 02, Epoch: 137, Loss: 0.2011, Train: 100.00%, Valid: 73.80%

Run: 03, Epoch: 33, Loss: 0.8655, Train: 100.00%, Valid: 69.60% Test: 73.00%
Run: 03, Epoch: 34, Loss: 0.8231, Train: 100.00%, Valid: 69.80% Test: 73.00%
Run: 03, Epoch: 35, Loss: 0.7979, Train: 100.00%, Valid: 70.20% Test: 73.10%
Run: 03, Epoch: 36, Loss: 0.7978, Train: 100.00%, Valid: 70.60% Test: 73.10%
Run: 03, Epoch: 37, Loss: 0.7788, Train: 100.00%, Valid: 70.40% Test: 73.10%
Run: 03, Epoch: 38, Loss: 0.7895, Train: 100.00%, Valid: 70.00% Test: 72.30%
Run: 03, Epoch: 39, Loss: 0.7812, Train: 100.00%, Valid: 69.60% Test: 72.20%
Run: 03, Epoch: 40, Loss: 0.7379, Train: 100.00%, Valid: 69.60% Test: 72.20%
Run: 03, Epoch: 41, Loss: 0.7314, Train: 100.00%, Valid: 69.40% Test: 72.20%
Run: 03, Epoch: 42, Loss: 0.6976, Train: 100.00%, Valid: 69.40% Test: 72.20%
Run: 03, Epoch: 43, Loss: 0.7116, Train: 100.00%, Valid: 69.20% Test: 71.60%
Run: 03, Epoch: 44, Loss: 0.7061, Train: 100.00%, Valid: 69.00% Test: 71.80%
Run: 03, Epoch: 45, Loss: 0.6874, Train: 100.00%, Valid: 69.20% Test: 72.00%

...

Run: 08, Epoch: 123, Loss: 0.2284, Train: 100.00%, Valid: 74.00% Test: 78.30%
Run: 08, Epoch: 124, Loss: 0.2261, Train: 100.00%, Valid: 74.00% Test: 78.30%
Run: 08, Epoch: 125, Loss: 0.2447, Train: 100.00%, Valid: 74.00% Test: 78.60%
Run: 08, Epoch: 126, Loss: 0.2097, Train: 100.00%, Valid: 74.20% Test: 78.60%
Run: 08, Epoch: 127, Loss: 0.2080, Train: 100.00%, Valid: 74.40% Test: 78.60%
Run: 08, Epoch: 128, Loss: 0.2238, Train: 100.00%, Valid: 74.60% Test: 78.70%
Run: 08, Epoch: 129, Loss: 0.2080, Train: 100.00%, Valid: 74.80% Test: 78.60%
Run: 08, Epoch: 130, Loss: 0.2330, Train: 100.00%, Valid: 74.40% Test: 78.50%
Run: 08, Epoch: 131, Loss: 0.2154, Train: 100.00%, Valid: 74.60% Test: 78.40%
Run: 08, Epoch: 132, Loss: 0.2143, Train: 100.00%, Valid: 74.60% Test: 78.50%
Run: 08, Epoch: 133, Loss: 0.2165, Train: 100.00%, Valid: 74.40% Test: 78.50%
Run: 08, Epoch: 134, Loss: 0.2280, Train: 100.00%, Valid: 74.40% Test: 78.50%
Run: 08, Epoch: 135, Loss: 0.1938, Train: 100.00%, Valid: 74.40%

Run: 09, Epoch: 28, Loss: 0.9245, Train: 100.00%, Valid: 69.40% Test: 73.20%
Run: 09, Epoch: 29, Loss: 0.8902, Train: 100.00%, Valid: 69.00% Test: 73.50%
Run: 09, Epoch: 30, Loss: 0.8738, Train: 100.00%, Valid: 69.00% Test: 73.60%
Run: 09, Epoch: 31, Loss: 0.8683, Train: 100.00%, Valid: 69.40% Test: 73.60%
Run: 09, Epoch: 32, Loss: 0.8390, Train: 100.00%, Valid: 69.80% Test: 73.90%
Run: 09, Epoch: 33, Loss: 0.8364, Train: 100.00%, Valid: 70.60% Test: 73.80%
Run: 09, Epoch: 34, Loss: 0.8204, Train: 100.00%, Valid: 71.40% Test: 74.30%
Run: 09, Epoch: 35, Loss: 0.7921, Train: 100.00%, Valid: 71.80% Test: 74.50%
Run: 09, Epoch: 36, Loss: 0.7756, Train: 100.00%, Valid: 72.60% Test: 74.70%
Run: 09, Epoch: 37, Loss: 0.7694, Train: 100.00%, Valid: 73.00% Test: 74.30%
Run: 09, Epoch: 38, Loss: 0.7622, Train: 100.00%, Valid: 73.60% Test: 74.60%
Run: 09, Epoch: 39, Loss: 0.7692, Train: 100.00%, Valid: 73.80% Test: 74.70%
Run: 09, Epoch: 40, Loss: 0.7480, Train: 100.00%, Valid: 74.00% Test: 74.90%

Run: 09, Epoch: 138, Loss: 0.1923, Train: 100.00%, Valid: 72.60% Test: 74.00%
Run: 09, Epoch: 139, Loss: 0.1646, Train: 100.00%, Valid: 72.80% Test: 74.00%
Run: 09, Epoch: 140, Loss: 0.1759, Train: 100.00%, Valid: 72.60% Test: 73.80%
Run: 09, Epoch: 141, Loss: 0.1585, Train: 100.00%, Valid: 72.80% Test: 73.60%
Run: 09, Epoch: 142, Loss: 0.1846, Train: 100.00%, Valid: 72.80% Test: 73.70%
Run: 09, Epoch: 143, Loss: 0.1711, Train: 100.00%, Valid: 72.80% Test: 73.90%
Run: 09, Epoch: 144, Loss: 0.1857, Train: 100.00%, Valid: 73.20% Test: 73.90%
Run: 09, Epoch: 145, Loss: 0.1811, Train: 100.00%, Valid: 73.00% Test: 73.80%
Run: 09, Epoch: 146, Loss: 0.1729, Train: 100.00%, Valid: 73.00% Test: 73.80%
Run: 09, Epoch: 147, Loss: 0.1871, Train: 100.00%, Valid: 72.40% Test: 73.70%
Run: 09, Epoch: 148, Loss: 0.1768, Train: 100.00%, Valid: 71.80% Test: 73.60%
Run: 09, Epoch: 149, Loss: 0.1641, Train: 100.00%, Valid: 72.20% Test: 73.90%
Run: 09, Epoch: 150, Loss: 0.1913, Train: 100.00%, Valid: 72.40%

Run: 10, Epoch: 43, Loss: 0.7357, Train: 100.00%, Valid: 76.00% Test: 77.70%
Run: 10, Epoch: 44, Loss: 0.7133, Train: 100.00%, Valid: 76.20% Test: 78.00%
Run: 10, Epoch: 45, Loss: 0.6986, Train: 100.00%, Valid: 76.40% Test: 78.00%
Run: 10, Epoch: 46, Loss: 0.6977, Train: 100.00%, Valid: 76.00% Test: 78.20%
Run: 10, Epoch: 47, Loss: 0.6692, Train: 100.00%, Valid: 76.20% Test: 78.30%
Run: 10, Epoch: 48, Loss: 0.6533, Train: 100.00%, Valid: 76.40% Test: 78.30%
Run: 10, Epoch: 49, Loss: 0.6698, Train: 100.00%, Valid: 76.20% Test: 78.00%
Run: 10, Epoch: 50, Loss: 0.6402, Train: 100.00%, Valid: 76.20% Test: 78.20%
Run: 10, Epoch: 51, Loss: 0.6408, Train: 100.00%, Valid: 76.40% Test: 78.30%
Run: 10, Epoch: 52, Loss: 0.6381, Train: 100.00%, Valid: 76.60% Test: 78.80%
Run: 10, Epoch: 53, Loss: 0.6089, Train: 100.00%, Valid: 77.00% Test: 78.70%
Run: 10, Epoch: 54, Loss: 0.5989, Train: 100.00%, Valid: 77.00% Test: 79.10%
Run: 10, Epoch: 55, Loss: 0.6015, Train: 100.00%, Valid: 76.80% Test: 78.80%

Run: 10, Epoch: 153, Loss: 0.1688, Train: 100.00%, Valid: 74.60% Test: 77.40%
Run: 10, Epoch: 154, Loss: 0.1607, Train: 100.00%, Valid: 74.60% Test: 77.80%
Run: 10, Epoch: 155, Loss: 0.1597, Train: 100.00%, Valid: 74.40% Test: 77.40%
Run: 10, Epoch: 156, Loss: 0.1541, Train: 100.00%, Valid: 74.40% Test: 77.20%
Run: 10, Epoch: 157, Loss: 0.1722, Train: 100.00%, Valid: 74.60% Test: 77.40%
Run: 10, Epoch: 158, Loss: 0.1667, Train: 100.00%, Valid: 74.20% Test: 77.40%
Run: 10, Epoch: 159, Loss: 0.1846, Train: 100.00%, Valid: 74.00% Test: 77.50%
Run: 10, Epoch: 160, Loss: 0.1563, Train: 100.00%, Valid: 74.20% Test: 77.40%
Run: 10, Epoch: 161, Loss: 0.1768, Train: 100.00%, Valid: 74.20% Test: 77.30%
Run: 10, Epoch: 162, Loss: 0.1652, Train: 100.00%, Valid: 74.20% Test: 77.20%
Run: 10, Epoch: 163, Loss: 0.1476, Train: 100.00%, Valid: 74.40% Test: 77.40%
Run: 10, Epoch: 164, Loss: 0.1747, Train: 100.00%, Valid: 74.40% Test: 77.20%
Run: 10, Epoch: 165, Loss: 0.1755, Train: 100.00%, Valid: 74.20%